# <font color = 'indianred'>**Identify Duplicate Questions in Quora Question Pairs using Siamese Network and Softmax** </font>

**Objective:**
In this notebook, we will built upon the prebvious notebook: Quora_find_duplicate_questions_bert.ipynb. We will understand how to train model using Siamese Network. In this notebook, we will use Huggingface Trainer.

**Plan**
1. Set Environment
2. Load Dataset
3. Accessing and Manipulating Splits
4. Load Pre-trained Tokenizer
5. Train Model
  1. Custom Data Collator <br>
  2. Download and modify the model config file <br>
  3. Custom Model Class <br>
  4. Training Arguments <br>
  5. Instantiate Trainer <br>
  6. Setup WandB <br>
  7. Training and Validation
6. Perfromance on Test Set
7. Model Inference



















# <font color = 'indianred'> **1. Setting up the Environment** </font>



In [ ]:
from pathlib import Path
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount("/content/drive")
    !pip install datasets transformers evaluate wandb accelerate swifter sentence-transformers -U -qq
    base_folder = Path("/content/drive/MyDrive/data")
else:
    base_folder = Path("/home/harpreet/Insync/google_drive_shaannoor/data")

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<font color = 'indianred'> *Load Libraries* </font>

In [ ]:
# standard data science libraries for data handling and visualization

import numpy as np
from sklearn.metrics.pairwise import paired_cosine_distances
from sklearn.metrics import precision_recall_fscore_support, accuracy_score



# New libraries introduced in this notebook
import evaluate
import torch
from datasets import load_dataset, DatasetDict, ClassLabel
from transformers import Pipeline
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import PreTrainedModel
# from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.modeling_outputs import ModelOutput
from transformers import BertModel, BertConfig
from sentence_transformers import util

import torch
import torch.nn as nn
import wandb

# <font color = 'indianred'> **2. Load Data set**
    


**Quora Dataset**

The Quora dataset is composed of question pairs, and the task is to determine if the questions are paraphrases of each other (have the same meaning).



In [ ]:
quora_dataset = load_dataset("quora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [ ]:
# Renaming 'is_duplicate' column to 'labels' to match the naming convention expected by Hugging Face Trainer
quora_dataset = quora_dataset.rename_column('is_duplicate', 'labels')

# Retrieve the features of the 'train' split from the quora_dataset
features = quora_dataset['train'].features

# Define the 'labels' feature as a ClassLabel with two classes: 'not_duplicate' and 'duplicate'
features['labels'] = ClassLabel(num_classes=2, names=['not_duplicate', 'duplicate'])

# Cast the 'labels' column in the dataset to the ClassLabel type, ensuring compatibility with Hugging Face's Trainer
quora_dataset = quora_dataset.cast(features)

Casting the dataset:   0%|          | 0/404290 [00:00<?, ? examples/s]

# <font color = 'indianred'> **3. Accessing and Manipulating Splits**</font>

We only have train split, we will create train/valid/test splits now.

<font color = 'indianred'>*Create futher subdivions of the splits*</font>

In [ ]:
# Split the test set into test and validation sets
train_temp_splits = quora_dataset["train"].train_test_split(
    test_size=0.3, seed=42)  # 70% for training, 30% for test/validation

val_test_splits = train_temp_splits["test"].train_test_split(
    test_size=0.5, seed=42)  # 15% for validation and 15% for test

# Extract the test and validation splits
train_split = train_temp_splits["train"]
valid_split = val_test_splits["train"]
test_split = val_test_splits["test"]


<font color = 'indianred'> *Create subset for experimentation* </font>

In [ ]:
train_split_small = train_split.shuffle(seed=42).select(range(10000))
val_split_small = valid_split.shuffle(seed=42).select(range(5000))
test_split_small = test_split.shuffle(seed=42).select(range(5000))

<font color = 'indianred'>*Combine splits*</font>

We will combine train and validation splits as we will be applying the same processing steps to both the splits.


In [ ]:
train_val_small = DatasetDict(
    {"train": train_split_small, "valid": val_split_small})

We have created the datset. The next step is to tokenize the dataset in a format so that we can pass the tokenized inputs to the pre-trained model.

# <font color = 'indianred'>**4. Load pre-trained Tokenizer**</font>

In our next step, we will download a pre-trained tokenizer specifically designed to work with BERT. This tokenizer will handle the conversion of our text into a format that BERT can understand.

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(batch):
  question1 = []
  question2 = []
  for question_pair in batch['questions']:
    question1.append(question_pair['text'][0])
    question2.append(question_pair['text'][1])

  tokenized_question1 = tokenizer(question1, truncation=True)
  tokenized_question2 = tokenizer(question2, truncation=True)
  return {
      'input_ids_q1': tokenized_question1['input_ids'],
      'attention_mask_q1': tokenized_question1['attention_mask'],
      'input_ids_q2': tokenized_question2['input_ids'],
      'attention_mask_q2': tokenized_question2['attention_mask'],
  }


In [ ]:
tokenized_dataset_small = train_val_small.map(tokenize_fn, batched=True).remove_columns( ['questions'])
tokenized_dataset_small.set_format(type='torch')
tokenized_dataset_small['train'].features

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'labels': ClassLabel(names=['not_duplicate', 'duplicate'], id=None),
 'input_ids_q1': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'attention_mask_q1': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_ids_q2': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'attention_mask_q2': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

#  <font color = 'indianred'> **5. Model Training**

##  <font color = 'indianred'> **5.1 Custom Data Collator**

In [ ]:
class SiameseDataCollatorWithPadding:
    def __init__(self, tokenizer, padding=True):
        """
        Custom data collator for Siamese network structure with separate tokenization for two inputs.

        Args:
        tokenizer (PreTrainedTokenizer): The tokenizer used for encoding the text inputs.
        padding (bool, optional): Whether to pad the inputs to the maximum length in the batch. Defaults to True.
        """
        self.tokenizer = tokenizer
        self.padding = padding

    def __call__(self, features):
        # Separate features for question1 and question2
        features_q1 = [{"input_ids": feature["input_ids_q1"], "attention_mask": feature["attention_mask_q1"]} for feature in features]
        features_q2 = [{"input_ids": feature["input_ids_q2"], "attention_mask": feature["attention_mask_q2"]} for feature in features]

        # Pad each set of features independently
        batch_q1 = self.tokenizer.pad(features_q1, padding=self.padding, return_tensors="pt")
        batch_q2 = self.tokenizer.pad(features_q2, padding=self.padding, return_tensors="pt")

        # Combine the padded features into one dictionary
        batch = {
            "input_ids_q1": batch_q1["input_ids"],
            "attention_mask_q1": batch_q1["attention_mask"],
            "input_ids_q2": batch_q2["input_ids"],
            "attention_mask_q2": batch_q2["attention_mask"],
        }

        # If labels exist, include them in the batch
        if "labels" in features[0]:
            batch["labels"] = torch.tensor([feature["labels"] for feature in features], dtype=torch.long)

        return batch


##  <font color = 'indianred'> **5.2 Downaload and Modify Model Config File**

<font color = 'indianred'>*Download config file of pre-trained Model*</font>



In [ ]:
config = BertConfig()
class_names = tokenized_dataset_small["train"].features["labels"].names
label2id = {label: i for i, label in enumerate(class_names)}
id2label = {i: label for label, i in label2id.items()}
config.id2label = id2label
config.label2id = label2id
config


BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "not_duplicate",
    "1": "duplicate"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "duplicate": 1,
    "not_duplicate": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

##  <font color = 'indianred'> **5.3 Custom Model Class**

In [ ]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(in_mask.sum(1), min=1e-9 )
    return pool

In [ ]:
class SiameseBertModel(PreTrainedModel):
    config_class = BertConfig

    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(config.hidden_size * 3, 2)  # Assuming binary classification (duplicate or not)

    def forward(self, input_ids_q1, attention_mask_q1, input_ids_q2, attention_mask_q2, labels=None):
        u = self.bert(input_ids_q1, attention_mask=attention_mask_q1).last_hidden_state
        v = self.bert(input_ids_q2, attention_mask=attention_mask_q2).last_hidden_state

        # print(u.size())
        # get the mean pooled vectors
        u = mean_pool(u, attention_mask_q1)
        v = mean_pool(v, attention_mask_q2)

        # build the |u-v| tensor
        uv = torch.sub(u, v)
        uv_abs = torch.abs(uv)

        # concatenate u, v, |u-v|
        x = torch.cat([u, v, uv_abs], dim=-1)
        logits = self.classifier(x)

        # compute the loss if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return ModelOutput(
            loss=loss.mean(),
            embeddings=(u,v)
        )


## <font color = 'indianred'> **5.4 Training Arguments**</font>







In [ ]:
# Define the directory where model checkpoints will be saved
model_folder = base_folder / "models"/"nlp_spring_2024/quora/sbert_hf_softmax_small"
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    # Training-specific configurations
    num_train_epochs=1,  # Total number of training epochs
    # Number of samples per training batch for each device
    per_device_train_batch_size=32,
    # Number of samples per evaluation batch for each device
    per_device_eval_batch_size=32,
    weight_decay=0.01,  # Apply L2 regularization to prevent overfitting
    learning_rate=2e-5,  # Step size for the optimizer during training
    optim='adamw_torch',  # Optimizer,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=50,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="eval_loss",
    greater_is_better=False,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=50,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='quora_sbert_hf_softmax_small',  # Experiment name for Weights & Biases


    fp16=True,
    remove_unused_columns=False
)


##  <font color = 'indianred'> **5.5 Initialize Trainer**</font>



In [ ]:
# initialize trainer
data_collator = SiameseDataCollatorWithPadding(tokenizer)
model = SiameseBertModel(config)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_small["train"],
    eval_dataset=tokenized_dataset_small["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## <font color = 'indianred'> **5.6 Setup WandB**</font>

In [ ]:
# setup wandb
wandb.login()  # you will need to craete wandb account first
# Set project name for logging
%env WANDB_PROJECT = nlp_course_spring_2024_quora-sbert-small

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=nlp_course_spring_2024_quora-sbert-small


##  <font color = 'indianred'> **5.7 Training and Validation**

In [ ]:
trainer.train()  # start training

wandb: Currently logged in as: hsingh-utd. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
50,0.629000,0.603595
100,0.591400,0.572828
150,0.562100,0.548231
200,0.536400,0.556873
250,0.560400,0.534980
300,0.533400,0.528812


TrainOutput(global_step=313, training_loss=0.5670179147689868, metrics={'train_runtime': 116.9198, 'train_samples_per_second': 85.529, 'train_steps_per_second': 2.677, 'total_flos': 0.0, 'train_loss': 0.5670179147689868, 'epoch': 1.0})

<font color = 'indianred'> *Evaluate model on Validation Set* </font>


In [ ]:
evaluation_results = trainer.evaluate(tokenized_dataset_small["valid"])
evaluation_results

{'eval_loss': 0.5288124680519104,
 'eval_runtime': 5.7778,
 'eval_samples_per_second': 865.374,
 'eval_steps_per_second': 27.173,
 'epoch': 1.0}

In [ ]:
wandb.log(evaluation_results)

<font color = 'indianred'> **Validation set accuracy, recall, f1 , precision** </font>

In [ ]:
eval_outputs = trainer.predict(tokenized_dataset_small["valid"])

In [ ]:
eval_outputs._fields

('predictions', 'label_ids', 'metrics')

In [ ]:
u, v = eval_outputs.predictions
labels = eval_outputs.label_ids

In [ ]:
scores = 1 - paired_cosine_distances(u, v)

In [ ]:
# the function is borrowed from the sentence-transformers library
def find_best_acc_and_threshold(scores, labels, high_score_more_similar: bool):
    assert len(scores) == len(labels)
    rows = list(zip(scores, labels))

    rows = sorted(rows, key=lambda x: x[0], reverse=high_score_more_similar)

    max_acc = 0
    best_threshold = -1

    positive_correct_so_far = 0 # positives predicted correctly so far
    negatives_correct_so_far = sum(labels == 0) # negatives predicted correctly so far

    for i in range(len(rows) - 1):
        score, label = rows[i]
        if label == 1:
            positive_correct_so_far += 1
        else:
            negatives_correct_so_far -= 1

        acc = (positive_correct_so_far + negatives_correct_so_far) / len(labels)
        if acc > max_acc:
            max_acc = acc
            best_threshold = (rows[i][0] + rows[i + 1][0]) / 2

    return max_acc, best_threshold

In [ ]:
eval_accuracy, threshold_accuracy = find_best_acc_and_threshold(scores, labels, True)

In [ ]:
eval_accuracy, threshold_accuracy

(0.717, 0.8474910259246826)

In [ ]:
wandb.log({"eval_accuracy": eval_accuracy, "sim_threshold_for_acc": threshold_accuracy})

In [ ]:
# the function is borrowed from the sentence-transformers library
def find_best_f1_and_threshold(scores, labels, high_score_more_similar: bool):
    assert len(scores) == len(labels)

    scores = np.asarray(scores)
    labels = np.asarray(labels)

    rows = list(zip(scores, labels))

    rows = sorted(rows, key=lambda x: x[0], reverse=high_score_more_similar)

    best_f1 = best_precision = best_recall = 0
    threshold = 0
    total_predicted_as_positives_so_far = 0
    true_positives_so_far = 0
    total_positives_in_data = sum(labels)

    for i in range(len(rows) - 1):
        score, label = rows[i]
        total_predicted_as_positives_so_far += 1

        if label == 1:
            true_positives_so_far += 1

        if true_positives_so_far > 0:
            precision = true_positives_so_far / total_predicted_as_positives_so_far
            recall = true_positives_so_far / total_positives_in_data
            f1 = 2 * precision * recall / (precision + recall)
            if f1 > best_f1:
                best_f1 = f1
                best_precision = precision
                best_recall = recall
                threshold = (rows[i][0] + rows[i + 1][0]) / 2

    return best_f1, best_precision, best_recall, threshold

In [ ]:
best_f1, best_precision, best_recall, threshold_f1 = find_best_f1_and_threshold(scores, labels, True)

In [ ]:
best_f1, best_precision, best_recall, threshold_f1

(0.6766948279414795,
 0.5532345013477089,
 0.8710875331564987,
 0.7654224038124084)

In [ ]:
wandb.log({"eval_f1": best_f1, "eval_precision": best_precision, "eval_recall": best_recall, "sim_threshold_for_f1": threshold_f1})

<font color = 'indianred'> *Get best checkpoint*</font>


In [ ]:
trainer.state.best_model_checkpoint

'/content/drive/MyDrive/data/models/nlp_spring_2024/quora/sbert_hf_softmax_small/checkpoint-300'

In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")


The best model was saved at step 300.


In [ ]:
wandb.finish()


epoch,▁
eval/loss,█▅▃▄▂▁▁
eval/runtime,▁▂▁▅▃▂█
eval/samples_per_second,█▇█▄▅▇▁
eval/steps_per_second,█▇█▄▅▇▁
eval_accuracy,▁
eval_f1,▁
eval_loss,▁
eval_precision,▁
eval_recall,▁
eval_runtime,▁


#  <font color = 'indianred'> **6. Performance on Test Set**


For test set evaluation, we will perfrom following steps:


1. **Load Model**: Retrieve the best-performing model checkpoint.
2. **Load Tokenizer**: Initialize the corresponding tokenizer for data preprocessing.
3. **Tokenize Test Data**: Convert the test set text data into a format suitable for the model using the tokenizer.
4. **Configure Training Arguments**: Set `TrainingArguments` with evaluation-specific configurations, disabling training and enabling evaluation.
5. **Initialize Trainer**: Create a `Trainer` instance with the model, training arguments, and evaluation dataset.
6. **Evaluate**: Use the `Trainer` instance to evaluate the model on the test set and obtain evaluation metrics.


<font color = 'indianred'>*Load model and tokenizer*</font>

In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
checkpoint

'/content/drive/MyDrive/data/models/nlp_spring_2024/quora/sbert_hf_softmax_small/checkpoint-300'

In [ ]:
model = SiameseBertModel.from_pretrained(checkpoint, config = config )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model.to('cuda')
model.eval()


SiameseBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
test_set_tokenized = test_split_small.map(tokenize_fn, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

<font color = 'indianred'>*Training Arguments*</font>



In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=128,
    do_train=False,
    do_eval=True,
    report_to=[],
    remove_unused_columns=False
)

<font color = 'indianred'>*Instantiate Trainer*</font>

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_set_tokenized,

    tokenizer=tokenizer,
    data_collator=data_collator,
)

<font color = 'indianred'>*Evaluate using Trainer*</font>

In [ ]:
test_outputs = trainer.predict(test_set_tokenized)

In [ ]:
u, v = test_outputs.predictions
scores = 1 - paired_cosine_distances(u, v)
labels = test_outputs.label_ids

In [ ]:
def evaluate_test(scores, threshold_acc, threshold_f1, labels):
    """
    Evaluate classification metrics based on similarity scores and separate thresholds
    for accuracy and for F1, precision, and recall.

    Args:
        scores (np.ndarray): Array of pairwise similarity scores.
        threshold_acc (float): Threshold for classifying pairs when calculating accuracy.
        threshold_f1 (float): Threshold for classifying pairs when calculating F1, precision, and recall.
        labels (np.ndarray): Ground truth binary labels indicating whether pairs are similar (1) or not (0).

    Returns:
        dict: Dictionary containing accuracy (based on threshold_acc) and F1 score, precision, and recall (based on threshold_f1).
    """
    # Convert scores to binary predictions based on the thresholds
    predictions_acc = (scores >= threshold_acc).astype(int)
    predictions_f1 = (scores >= threshold_f1).astype(int)

    # Compute accuracy using the threshold for accuracy
    accuracy = accuracy_score(labels, predictions_acc)

    # Compute precision, recall, and F1 score using the threshold for F1, precision, and recall
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions_f1, average='binary')

    return {
        "test_accuracy": accuracy,
        "test_f1_score": f1,
        "test_precision": precision,
        "test_recall": recall
    }




In [ ]:
test_evaluation = evaluate_test(scores, threshold_acc=threshold_accuracy, threshold_f1=threshold_f1, labels=labels)

In [ ]:
test_evaluation

{'test_accuracy': 0.7126,
 'test_f1_score': 0.6598639455782314,
 'test_precision': 0.5346193592834998,
 'test_recall': 0.8617434758467518}

# <Font color = 'indianred'> **7. Model Inference**
Inference is generally performed on unlabeled datasets, reflecting real-world applications where true labels are unknown. To replicate this, we'll exclude the 'labels' column from our dataset. Moreover, considering the Quora dataset's distinctive arrangement of IDs and question pairs, we'll reformat it to match the typical structure of sentence pair datasets. This standard format usually consists of 'sentence1', 'sentence2', and optionally 'labels' (for labeled datasets), ensuring compatibility with common model expectations and facilitating more seamless data handling and model inference.

In [ ]:
class SentenceEmbeddingPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        return {}, {}, {}

    def preprocess(self, text, second_text=None):
        return self.tokenizer(text, return_tensors=self.framework, padding = True, truncation = True)

    def _forward(self, model_inputs):
        token_embeds= self.model(**model_inputs).last_hidden_state
        attention_mask = model_inputs['attention_mask']
        in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
        pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(in_mask.sum(1), min=1e-9 )

        return pool

    def postprocess(self, model_outputs):
        return model_outputs

In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
model = SiameseBertModel.from_pretrained(checkpoint, config = config ).bert
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
sentence_pair_pipeline = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer, device = 0, framework='pt')

In [ ]:
sentences = ['What do House Republicans think of President Obama?',
 'Do republicans really think President Obama did a bad job?',
 'Why are so many people content with just earning a salary and working 9-6 their entire adult life?',
 'Jobs and Careers: Why are so many people content with just earning a salary and working 9-6 their entire adult life?',
 'How do you check the balance on a target gift card?',
 'How do you check your balance on a Target gift card?',
 'What are the best tips to stay young looking?',
 'What are best ways to stay and look young for longer time?',
 'How do you go about writing a novel?',
 'What are some tips for writing a novel?',
 'Is downloading app slow down the WiFi?',
 'Why does Xbox slow down when downloading games? Is there any setting to improve its speed?',
 'What is a good website for free books?',
 'Where can I get online PDF or EPUB versions of books?',
 'How do you switch phones on Metro PCS?',
 'How can I switch from Sprint to Metro PCs?',
 "Why don't some people fear death?",
 'Why do people fear death?',
 'What is the weighted average income in the United States?',
 'How does jumping rope help burn fat?']



In [ ]:
embeddings = sentence_pair_pipeline(sentences)

In [ ]:
embeddings = torch.cat(embeddings, dim=0)

In [ ]:
# Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)
cos_sim


tensor([[1.0000, 0.8113, 0.3257, 0.3601, 0.3106, 0.3047, 0.5021, 0.4440, 0.4418,
         0.4954, 0.3872, 0.4225, 0.4497, 0.3968, 0.3570, 0.3590, 0.4774, 0.4905,
         0.4412, 0.4321],
        [0.8113, 1.0000, 0.3277, 0.3346, 0.2821, 0.2774, 0.4098, 0.3956, 0.3565,
         0.3823, 0.3994, 0.4509, 0.3908, 0.3810, 0.3036, 0.3243, 0.4853, 0.4729,
         0.3975, 0.3947],
        [0.3257, 0.3277, 1.0000, 0.9422, 0.3723, 0.3725, 0.3872, 0.4594, 0.3241,
         0.2490, 0.3317, 0.4016, 0.2961, 0.2948, 0.3393, 0.2958, 0.3155, 0.3413,
         0.4850, 0.3491],
        [0.3601, 0.3346, 0.9422, 1.0000, 0.3699, 0.3741, 0.4263, 0.4656, 0.3479,
         0.2876, 0.3715, 0.4110, 0.3153, 0.3093, 0.3615, 0.2930, 0.2708, 0.2996,
         0.5324, 0.3793],
        [0.3106, 0.2821, 0.3723, 0.3699, 1.0000, 0.9757, 0.4605, 0.3178, 0.4437,
         0.3966, 0.3665, 0.3815, 0.4522, 0.4317, 0.5474, 0.3374, 0.2987, 0.3469,
         0.4229, 0.4002],
        [0.3047, 0.2774, 0.3725, 0.3741, 0.9757, 1.0000, 0.4

In [ ]:
# Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim) - 1):
    for j in range(i + 1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

# Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))


Top-5 most similar pairs:
How do you check the balance on a target gift card? 	 How do you check your balance on a Target gift card? 	 0.9757
Why are so many people content with just earning a salary and working 9-6 their entire adult life? 	 Jobs and Careers: Why are so many people content with just earning a salary and working 9-6 their entire adult life? 	 0.9422
Why don't some people fear death? 	 Why do people fear death? 	 0.9335
What is a good website for free books? 	 Where can I get online PDF or EPUB versions of books? 	 0.8125
What do House Republicans think of President Obama? 	 Do republicans really think President Obama did a bad job? 	 0.8113
